# Semantic Relationship Extraction and Feature Engineering

By christian Spiteri Gauci and Adam Darmanin

## Literature and References

## Method

* Cleaning of data - Remove special characters, punctuation, and extra whitespace
* Tokenise using spacy
* Lemmatize tokens and remove stopwords
* extract relationships - using "nsubj", "ROOT", "dobj"
* Clean the extracted relationships - remove any entries that inlcude only special characters or numbers
* Remove entries that occur only once - and keep those that occur 3times or more in each subreddit 
* Remove entries that occur in more than one subreddit 


In [1]:
import pandas as pd
import numpy as np
import spacy
from collections import defaultdict, Counter
import re
import concurrent.futures


import ast
from tqdm import tqdm
import nltk
from nltk.corpus import words as nltk_words


from lib.sanitze_util import clean_text_batch

tqdm.pandas()  # for progressbase in DFs.

nlp = spacy.load("en_core_web_sm")
nltk.download("words")
nltk.download('vader_lexicon')


english_words = set(nltk_words.words())

C:\Users\adamd\AppData\Local\Temp\ipykernel_11756\1435294090.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamd\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\adamd\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Data Preprocessing

1. Clean stopwords
2. Lemmatize
3. Convert emoticons
4. Expand contractions

In [2]:
df = pd.read_csv("./data/mental_disorders_reddit.csv")

# Create a new DataFrame with specific columns
columns_needed = ["title", "selftext", "subreddit"]
df_subset = df[columns_needed].copy()

# Drop anything not a full peice of text in this dataset.
mask = df_subset.applymap(lambda x: isinstance(x, str) and x.strip() != "")
df_subset = df_subset[mask["title"] & mask["selftext"]]
df_subset["raw_title"] = df_subset["title"]
df_subset["raw_selftext"] = df_subset["selftext"]
df_subset["title"] = clean_text_batch(df_subset["title"].fillna("").tolist())
df_subset["selftext"] = clean_text_batch(df_subset["selftext"].fillna("").tolist())

C:\Users\adamd\AppData\Local\Temp\ipykernel_11756\2031721653.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df_subset.applymap(lambda x: isinstance(x, str) and x.strip() != "")
Cleaning Pipeline Token:   0%|          | 0/738 [00:00<?, ?it/s]c:\Users\adamd\workspace\mining-large-scale-data\lib\sanitze_util.py:88: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
c:\Users\adamd\.conda\envs\gnn\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Cleaning Pipeline Token: 100%|██████████| 738/738 [00:02<00:00, 322.69it/s]


# Feature Engineering

1. Semantic relations between words
2. Mental health labels in tokens

In [3]:
from lib.feature_engineering import create_rel_feature,create_psylabel_feature,create_sentiment_feature,create_emotional_categories_scores_feature
tqdm.pandas()

df_subset["psy_labels"]=df_subset.progress_apply(create_psylabel_feature, axis=1)
df_subset["sentiment"]=df_subset.progress_apply(create_sentiment_feature, axis=1)
df_subset["emotional_categories"]=df_subset.progress_apply(create_emotional_categories_scores_feature, axis=1)
df_subset["semantic_relationships"]=df_subset.progress_apply(create_rel_feature, axis=1)

# Cache it for further processing down the line.
df_subset.to_csv("./data/SemanticsRel.csv", index=False)
df_subset[
    [
        "title",
        "selftext",
        "psy_labels",
        "sentiment",
        "emotional_categories",
        "semantic_relationships",
    ]
].head(15)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamd\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\adamd\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|██████████| 738/738 [00:11<00:00, 63.92it/s]


,title,selftext,psy_labels,sentiment,emotional_categories,semantic_relationships
0,life pointless,think important life relationship like absolut...,{},0.9501,"{'help': 0.0, 'violence': 0.0, 'sleep': 0.0, '...","[(ask, nsubj, relationship), (ask, dobj, goals)]"
1,cold rage,hello fellow friends bpd spectrum discouraged ...,{'SYMPTOMS': ['anger']},-0.9840,"{'help': 0.01, 'violence': 0.01, 'sleep': 0.01...","[(discouraged, nsubj, spectrum), (discouraged,..."
2,know,f20 bf m20 told today said wish better likes n...,{},0.8126,"{'help': 0.0, 'violence': 0.0, 'sleep': 0.0, '...","[(told, nsubj, m20), (said, nsubj, think), (kn..."
3,help opinions advice,okay open things past proud person anymore sto...,{'SYMPTOMS': ['anxiety']},0.9919,"{'help': 0.02, 'violence': 0.02, 'sleep': 0.0,...","[(help, dobj, opinions), (help, dobj, advice),..."
4,help,removed,{},0.4019,"{'help': 0.5, 'violence': 0.0, 'sleep': 0.0, '...",[]
5,ex got diagnosed bpd,going detail diagnosis explains sudden break u...,{},0.9611,"{'help': 0.03, 'violence': 0.03, 'sleep': 0.0,...","[(going, dobj, diagnosis), (explains, dobj, up..."
6,misdiagnosis bpd common asking diagnosed years...,reposting larger sub recommendation people sma...,{'NEURO-DEVELOPMENTAL DISORDERS': ['autism']},0.9231,"{'help': 0.0, 'violence': 0.01, 'sleep': 0.0, ...","[(think, nsubj, bpd), (asking, dobj, years), (..."
7,trouble identifying sexual orientation bpd lik...,grew dating men realized teenager like women k...,{},0.9349,"{'help': 0.0, 'violence': 0.03, 'sleep': 0.0, ...","[(likes, nsubj, trouble), (identifying, dobj, ..."
8,needing advice,posted sub earlier today having trouble believ...,{'SYMPTOMS': ['anxiety']},-0.7003,"{'help': 0.0, 'violence': 0.02, 'sleep': 0.0, ...","[(needing, dobj, advice), (posted, dobj, sub),..."
9,bpd,removed,{},0.0000,"{'help': 0.0, 'violence': 0.0, 'sleep': 0.0, '...",[]


# Create the Nodes

Nodes have edges, attributes for MH disorders, wieghts and sentiment.

In [4]:
clean_data = []

for idx, row in tqdm(df_subset.iterrows()):
    subreddit = row["subreddit"]
    labels = row["psy_labels"]
    relationships = row["semantic_relationships"]
    title = row["title"]
    selftext = row["selftext"]
    for rel in relationships:
        if len(rel) == 3:  # Ensure the tuple has three elements
            word1, dep, word2 = rel
            clean_data.append(
                {
                    "Subreddit": subreddit,
                    "Word1": word1,
                    "Dependency": dep,
                    "Word2": word2,
                    "MHlabels": labels,
                    "title": title,
                    "selftext": selftext,
                }
            )
        else:
            print(f"Issue with relationship: {rel}")

clean_df = pd.DataFrame(clean_data)
clean_df.to_csv("./data/GraphData.csv", index=False)
clean_df.head(15)

738it [00:00, 4953.01it/s]


,Subreddit,Word1,Dependency,Word2,MHlabels,title,selftext
0,BPD,ask,nsubj,relationship,{},life pointless,think important life relationship like absolut...
1,BPD,ask,dobj,goals,{},life pointless,think important life relationship like absolut...
2,BPD,discouraged,nsubj,spectrum,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
3,BPD,discouraged,dobj,characteristics,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
4,BPD,wondering,nsubj,levels,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
5,BPD,experiencing,dobj,anger,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
6,BPD,found,nsubj,way,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
7,BPD,found,dobj,blame,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
8,BPD,find,nsubj,understanding,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...
9,BPD,tend,nsubj,ones,{'SYMPTOMS': ['anger']},cold rage,hello fellow friends bpd spectrum discouraged ...


In [5]:
# Grouping the DataFrame by 'Subreddit', 'Word1', 'Dependency', and 'Word2' and counting occurrences
grouped = (
    clean_df.groupby(["Subreddit", "Word1", "Dependency", "Word2"])
    .size()
    .reset_index(name="Occurrence")
)

# Filter out entries where Word1 and Word2 are not in the English dictionary
# filtered_relationships = multiple_subreddit_relationships[
#     (multiple_subreddit_relationships['Word1'].isin(english_words)) &
#     (multiple_subreddit_relationships['Word2'].isin(english_words))
# ]

# Define a regular expression pattern to match only alphanumeric words
pattern = re.compile(r"^[a-zA-Z]+$")

# Filter out entries where Word1 and Word2 contain only alphanumeric characters
filtered_relationships = grouped[
    (grouped["Word1"].str.match(pattern)) & (grouped["Word2"].str.match(pattern))
]

# Filtering relationships occurring more than three times within each subreddit
common_relationships = filtered_relationships[filtered_relationships["Occurrence"] > 3]


# Grouping the DataFrame by 'Word1', 'Dependency', and 'Word2' to count unique occurrences across subreddits
relationship_counts = (
    common_relationships.groupby(["Word1", "Dependency", "Word2"])
    .agg({"Subreddit": "nunique"})
    .reset_index()
)

# Filter relationships occurring in only one subreddit
single_subreddit_relationships = relationship_counts[
    relationship_counts["Subreddit"] == 1
]

# Merge to keep only the relationships occurring in one subreddit from common_relationships
filtered_common_relationships = pd.merge(
    common_relationships,
    single_subreddit_relationships,
    on=["Word1", "Dependency", "Word2"],
    how="inner",
)

print(filtered_common_relationships)
filtered_common_relationships.to_csv("SemanticsRelFiltered.csv", index=False)

   Subreddit_x        Word1 Dependency         Word2  Occurrence  Subreddit_y
0          BPD    diagnosed       dobj           bpd           9            1
1          BPD         feel       dobj          love           4            1
2          BPD         feel      nsubj           bpd           4            1
3          BPD         feel      nsubj          help           4            1
4          BPD         feel      nsubj        issues           5            1
5          BPD         feel      nsubj          know           4            1
6          BPD         feel      nsubj          life           4            1
7          BPD         feel      nsubj          love           4            1
8          BPD         feel      nsubj        people           8            1
9          BPD         feel      nsubj  relationship           4            1
10         BPD         feel      nsubj          talk           4            1
11         BPD         feel      nsubj        things           4

Some entries have word1 and word2 which are not actual words but numbers or mispelled words. Even though there was a chance that these were to be filtered due to occurrence should be more than 3, some weren't. At this point i was going to use the NLTK library to check for english words, but upon implementing, some important words like 'zyprexa' was eliminated since it's not included in the corpus. However this is an important entry and therefore should remain. a more crude filter, removing numbers only entries was implemented.

In [6]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
"zyprexa" in english_vocab

False